In [37]:
import pandas as pd
import pandas_ta as ta
import talib as TA
from backtesting import Strategy, Backtest
from datetime import datetime
import matplotlib.pyplot as plt
import plotly.graph_objects as go

LOAD AND CLEAN DATA

In [25]:
df = pd.read_csv('data_sets/EURUSD_Candlestick_4_Hour_BID_01.03.2014-02.04.2022.csv')
df = df[df['Volume']!=0]
df.isna().sum()
df.reset_index(drop=True, inplace=True)
df.tail()

,Local time,Open,High,Low,Close,Volume
12603,31.03.2022 20:00:00.000 GMT-0500,1.10673,1.10730,1.10613,1.10637,21587.74
12604,01.04.2022 00:00:00.000 GMT-0500,1.10638,1.10757,1.10428,1.10605,55565.31
12605,01.04.2022 04:00:00.000 GMT-0500,1.10605,1.10677,1.10365,1.10438,53602.25
12606,01.04.2022 08:00:00.000 GMT-0500,1.10438,1.10536,1.10279,1.10402,28882.14
12607,01.04.2022 12:00:00.000 GMT-0500,1.10402,1.10524,1.10325,1.10427,15225.47


CALCULATION OF ATR FOR SL & TP

In [26]:
df['ATR']= df.ta.atr()
df.tail()

,Local time,Open,High,Low,Close,Volume,ATR
12603,31.03.2022 20:00:00.000 GMT-0500,1.10673,1.10730,1.10613,1.10637,21587.74,0.003656
12604,01.04.2022 00:00:00.000 GMT-0500,1.10638,1.10757,1.10428,1.10605,55565.31,0.003630
12605,01.04.2022 04:00:00.000 GMT-0500,1.10605,1.10677,1.10365,1.10438,53602.25,0.003593
12606,01.04.2022 08:00:00.000 GMT-0500,1.10438,1.10536,1.10279,1.10402,28882.14,0.003520
12607,01.04.2022 12:00:00.000 GMT-0500,1.10402,1.10524,1.10325,1.10427,15225.47,0.003411


PLOTTING WHOLE DATASET

In [45]:
dfpl = df[:]
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                   open=dfpl['Open'],
                   high=dfpl['High'],
                   low=dfpl['Low'],
                   close=dfpl['Close'])])
fig.show()

CALCULATING SUPPORT & RESISTANCE

In [27]:
def support(df, l, n1, n2):
    # checking if the lows before "l" are decreasing
    for i in range(l-n1+1, l+1):
        if df.Low[i] > df.Low[i-1]:
            return 0
    # checking if the lows after "l" are increasing    
    for i in range(l+1, l+n2+1):
        if df.Low[i] < df.Low[i-1]:
            return 0
    
    return 1
        

def resistance(df, l, n1, n2):
    # checking if the highs before "l" are decreasing
    for i in range(l-n1+1, l+1):
        if df.High[i] < df.High[i-1]:
            return 0
    # checking if the highs after "l" are increasing    
    for i in range(l+1, l+n2+1):
        if df.High[i] > df.High[i-1]:
            return 0
    
    return 1

CHECK FOR SUPPORT & RESISTANCE LEVELS

In [35]:
ss = []
rr = []
n1 = 2
n2 = 2

for row in range(2, 12605):
    if support(df, row, n1, n2):
        ss.append((row, df.Low[row]))
    if resistance(df, row, n1, n2):
        rr.append((row, df.High[row]))

BACKTEST USING ATR FOR SL & TP

In [4]:
class SRCP(Strategy):
    
    def init(self):
        super().init()
        self.engulfing = self.I(TA.CDLENGULFING, self.data.Open, self.data.High, self.data.Low, self.data.Close, penetration=0)
        self.sstar = self.I(TA.CDLSHOOTINGSTAR, self.data.Open, self.data.High, self.data.Low, self.data.Close)
        self.hammer = self.I(TA.CDLHAMMER, self.data.Open, self.data.High, self.data.Low, self.data.Close)
        self.sr = self.I(pass)
    
    def next(self):
        super().next()
        pass